# Stopwords Austria

In [1]:
# Data processing
import json
import numpy as np
import pandas as pd
import re

# Text preprocessiong
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.corpus import stopwords

#getting german stopwords:
german_stop_words = stopwords.words('german')

# Funcoes para limpar as palavras:

def remove_stopwords(element):
    if isinstance(element, str):
        if element in german_stop_words:
            return ''
        else:
            return element

def replace_char(x):
  return re.sub('[^a-zA-Z\süöäß]', '', x)

def clean_corpus(s):
    s = s.apply(lambda x: x.lower() if isinstance(x, str) else x)     #tolower:
    idx = s.apply(lambda x: isinstance(x, str))                       # Criar um índice booleano para filtrar os valores não-caracteres
    s = s.drop(idx[~idx].index)                                       # Remover os valores não-caracteres
    s = s.apply(replace_char)                                         # remover as partes de palavras que nao sao caracteres ou espacos
    idx = s.apply(lambda x: len(x) < 2)                               # Criar um índice booleano para filtrar os elementos com comprimento menor que 2
    s = s.drop(idx[idx].index)                                        # Remover os elementos com comprimento menor que 2
    s = s.apply(remove_stopwords)                                     # remover as stopwords: 
    idx = s.apply(lambda x: x == '')                                  # Criar um índice booleano para filtrar os elementos vazios
    s = s.drop(idx[idx].index)                                        # Remover os elementos vazios
    
    return s

def frequency_words(x):
    # Cria um dicionário vazio para armazenar os contadores
    contadores = {} 
    
    for palavra in x:
        if palavra in contadores:                   # Se a palavra já estiver no dicionário, incrementa o contador
            contadores[palavra] += 1
        else:                                       # Se a palavra não estiver no dicionário, adiciona-a e define o contador como 1
            contadores[palavra] = 1

    return contadores

# Abrindo o arquivo:

df_original = pd.read_json('C:\\Users\\opc\\Documents\\Python_Scripts\\new_champions\\01_gesetze_sammlung\\at_concatenado.json')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\opc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\opc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\opc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Considerations

### Stopwords

I'm going to separate the documents per thema, as I did for the european Stopwords.
I will add to stopwords:

1. Stopwords that refers to Austria
2. Pronouns that might scape the original stopword list

Since we are not working with laws, I don'T expect that terms related to law (i.e. "Absatzt") will play a role here

### Document division

I'm going to divide the Austrian and German normative documents as:

- Daten and KI documents
- Timber and Circular economy / Green Deal documents
- Regierungsprogramm

The reason for that is that the last one has no specific thema, but should mention all the government's subthemas.

In [2]:
#df_original['documento'].unique()

#'digitale_aktionsplan_at', 'nationale_ki_strategie_2023_addendum_at', 'nationale_ki_strategie_2023', 'nationale_strategie_klimaziele',
df_daten_ki_at = df_original.query("documento == 'digitale_aktionsplan_at' | documento == 'nationale_ki_strategie_2023_addendum_at' | documento == 'nationale_ki_strategie_2023' | documento == 'nationale_strategie_klimaziele' ")

# 'regierungsprogramm' Onde coloco o regiuerungsprogramme? Que falaria dos dois?
df_reg_at = df_original.query("documento == 'regierungsprogramm'")

# timber and klima : 'oestereiche_holzinitiative', 'nationale_strategie_kreislaufwirtschaft'
df_kl_timber_at = df_original.query("documento == 'oestereiche_holzinitiative' | documento == 'nationale_strategie_kreislaufwirtschaft' ")

In [4]:
### 1. Olhando a frequência das palavras para incrementar as stopwords:
# DATA AND KI #

# Concatenando todas as linhas e depois criando uma série com as palavras
# lembrando que AQUI eu quero só a frequencia das palavras, por isso elas podem compor a mesma série
s = pd.Series(df_daten_ki_at['corpus'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

# criando um dicionário compreensao:
most_common = {k: v for k, v in freq.items() if v > 100}

#most_common

# len(german_stop_words) 232

232

In [5]:
#konjunktionen und pronomen, adverben
prep = ['et' , 'al' ,'sowohl' ,'hinsichtlich' ,'insbesondere' ,'jedoch' ,'etc', 'aufgrund' ,'derzeit', 'bereits', 'etwa', 'dabei', 'daher', 'sowie', 'bzw', 'jedoch', 'gegenüber', 'deren', 'neben', 'besonders', 'weitere' 'zb']

# redundante
at_woerte = ['österreichische' ,'österreich','österreichischen' , 'bundesregierung']

german_stop_words.extend(prep)
german_stop_words.extend(at_woerte)

# len(prep) + len (at_woerte) 25
# 232 + 25

# len(german_stop_words) # 257

In [10]:
# Revisando
s = pd.Series(df_daten_ki_at['corpus'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

for key, value in sorted(freq.items(), key=lambda item: item[1], reverse=True):
    print(key, value)

# Acho que está ok! Agora tenho que ver isso no topic modeling mesmo.

maßnahmen 604
umsetzung 545
anpassung 542
klimawandel 507
aktivitätsfelder 380
handlungsempfehlunge 374
auswirkungen 370
klimawandels 369
entwicklung 341
bundesländer 318
ki 288
berücksichtigung 287
weitere 280
bereich 267
schutz 267
rahmen 260
bmlfuw 257
ziel 250
bestehenden 244
naturgefahren 234
bund 232
daten 225
bedarf 222
wurde 216
klima 214
gemeinden 211
zb 210
besteht 208
ressourcen 207
raumordnung 206
bezug 205
forschung 201
verbesserung 191
link 191
aktivitätsfeldern 189
nutzung 187
instrumenten 180
gesundheit 178
verstärkte 177
forcierung 172
landwirtschaft 168
aktivitätsfeld 168
einsatz 166
schritte 165
unternehmen 159
tourismus 159
wurden 158
eu 156
energie 152
bedeutung 152
wirtschaft 150
anpassungsmaßnahmen 147
bevölkerung 146
sollen 144
erhöhung 142
kapitel 141
informationen 139
sicherung 139
potenzial 138
folgen 135
feldern 135
zeithorizont 134
tragende 134
wasserwirtschaft 132
planung 132
aktivitäts 131
bauen 130
förderung 129
konflikt 129
arten 128
schaffung 126
mensc

In [11]:
# Testando com o segundo banco:

# Clima #

# Concatenando todas as linhas e depois criando uma série com as palavras
# lembrando que AQUI eu quero só a frequencia das palavras, por isso elas podem compor a mesma série
s = pd.Series(df_kl_timber_at['corpus'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

for key, value in sorted(freq.items(), key=lambda item: item[1], reverse=True):
    print(key, value)

fur 313
kreislaufwirtschaft 169
nachhaltige 85
osterreich 63
produkte 62
holz 61
maßnahmen 59
osterreichische 59
sollen 52
kreislaufwirtschaftsstrategie 47
nutzung 45
zirkulare 45
zb 44
entwicklung 42
transformation 42
neue 40
holzinitiative 37
unternehmen 36
wirtschaft 34
uber 34
produkten 34
ziele 33
umsetzung 33
nachhaltigen 32
osterreichischen 31
materialien 30
schritte 30
nachsten 28
ziel 27
recycling 27
moglichst 27
gilt 26
dienstleistungen 26
rohstoffe 25
beschaffung 25
ressourcen 24
genutzt 24
textilien 24
rahmenbedingungen 23
pro 23
circular 23
klimaschutz 22
rahmen 22
europaischen 22
konnen 22
einsatz 21
verwendung 20
forschung 20
sinne 20
jahr 20
akteurinnen 20
konsumentinnen 20
bedeutung 19
beitrag 19
nachfrage 19
offentliche 19
green 19
fordern 19
elektro 19
seite 18
bmk 18
bereich 18
zusammenarbeit 18
produktion 18
schaffen 18
geschaftsmodelle 18
zirkularen 18
siehe 18
starken 18
eu 18
wiederverwendung 18
innovation 17
technologien 17
dafur 17
aktionsplan 17
gestaltung 16

In [12]:
# redundante
new_woerte = ['fur' , 'osterreich' , 'osterreichische', 'zb' , 'uber', 'osterreichischen']

german_stop_words.extend(new_woerte) 

# len(german_stop_words) 263 OK

In [14]:
# Testando com o terceiro banco:

# regierung #

# Concatenando todas as linhas e depois criando uma série com as palavras
# lembrando que AQUI eu quero só a frequencia das palavras, por isso elas podem compor a mesma série
s = pd.Series(df_reg_at['corpus'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

for key, value in sorted(freq.items(), key=lambda item: item[1], reverse=True):
    print(key, value)

prüfung 194
stärkung 165
maßnahmen 161
weiterentwicklung 138
ausbau 136
ziel 129
umsetzung 128
schaffung 122
evaluierung 122
sollen 109
bereich 107
menschen 103
berücksichtigung 100
entwicklung 96
zusammenarbeit 91
förderung 90
einsatz 90
rahmen 88
unternehmen 85
unterstützung 85
mehr 82
europäischen 78
öffentlichen 76
rahmenbedingungen 72
stärken 72
bestehenden 71
neue 67
einführung 66
ebene 64
österreichs 62
ermöglichen 61
verbesserung 61
herausforderungen 60
basis 60
schaffen 59
möglichkeit 58
digitalisierung 58
frauen 58
schutz 57
erhöhung 57
sicherstellung 57
wirtschaft 56
bundes 56
zukunft 55
europäischer 52
müssen 51
ländern 51
verstärkte 51
nutzung 51
sicherheit 50
dafür 50
bekennt 49
eu 48
land 47
neuen 46
integration 46
sinne 46
fokus 46
schulen 46
verwaltung 45
gemeinden 45
forschung 45
überprüfung 44
einbeziehung 43
erarbeitung 43
fördern 43
unserer 42
bildung 41
reduktion 41
nachhaltige 40
transparenz 40
europa 39
gemeinsame 39
bund 39
ausweitung 39
kinder 38
digitale 38
i

In [15]:
# add adverbio:

german_stop_words.extend(['mehr']) 

In [16]:
with open("austrian_stopwords.json", "w") as f:
    json.dump(german_stop_words, f)

# # Carregue a lista a partir do arquivo
# with open("lista.json", "r") as f:
#     lista_carregada = json.load(f)  